## Testing embeddings

In [ ]:
import numpy as np
from openai import AsyncOpenAI


def euclidean_distance(embedding1, embedding2):
    """Calculate Euclidean distance between embeddings"""
    emb1 = np.array(embedding1)
    emb2 = np.array(embedding2)
    return np.linalg.norm(emb1 - emb2)


def cosine_similarity_raw(embedding1, embedding2):
    """Calculate raw cosine similarity between embeddings (-1 to 1 range)"""
    emb1 = np.array(embedding1)
    emb2 = np.array(embedding2)

    # Calculate cosine similarity
    dot_product = np.dot(emb1, emb2)
    norm1 = np.linalg.norm(emb1)
    norm2 = np.linalg.norm(emb2)
    return dot_product / (norm1 * norm2)


def cosine_similarity(embedding1, embedding2):
    """Calculate cosine similarity for APIM comparison - likely uses raw cosine similarity"""
    # APIM probably uses raw cosine similarity without the [0,1] conversion
    raw_sim = cosine_similarity_raw(embedding1, embedding2)

    # If raw similarity is negative or very low, APIM might use a different approach
    # Let's try using only the positive part and see if it matches better
    return max(0, raw_sim)


def cosine_distance(embedding1, embedding2):
    """Calculate cosine distance (1 - cosine_similarity)"""
    sim = cosine_similarity_raw(embedding1, embedding2)
    return 1 - sim


def apim_style_distance(similarity):
    """Calculate distance the way APIM does: distance = 1 - similarity"""
    return 1 - similarity


def distance_to_similarity(distance, max_distance=None):
    """Convert distance to similarity score"""
    if max_distance is None:
        # For normalized embeddings, max distance is typically 2
        max_distance = 2.0

    # Similarity = 1 - (distance / max_distance)
    similarity = 1 - (distance / max_distance)
    return max(0, similarity)  # Ensure non-negative


def similarity_from_embeddings(embedding1, embedding2):
    """Calculate similarity score from embeddings"""
    distance = euclidean_distance(embedding1, embedding2)
    return distance_to_similarity(distance)


async def get_embedding(text, client: AsyncOpenAI, model="text-embedding-3-small"):
    """Get embedding for a text string"""
    response = await client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

In [ ]:
# Get embeddings from the model
import os
from semantic_kernel.agents import (
    AzureAIAgent,
    AzureAIAgentSettings,
)
from azure.identity import DefaultAzureCredential, AzureDeveloperCliCredential
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv(override=True)

endpoint = os.environ.get("AZURE_AI_FOUNDRY_CONNECTION_STRING")
deployment_name = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION", None)
tenant_id = os.environ.get("AZURE_TENANT_ID", None)

ai_agent_settings = AzureAIAgentSettings(
    endpoint=endpoint,
    model_deployment_name=deployment_name,
    api_version=api_version,
)
print(ai_agent_settings)

creds = (
    AzureDeveloperCliCredential(tenant_id=tenant_id)
    if os.environ.get("USE_AZURE_DEV_CLI") == "true"
    else DefaultAzureCredential()
)

client = AzureAIAgent.create_client(
    credential=creds,
    endpoint=ai_agent_settings.endpoint,
    api_version=ai_agent_settings.api_version,
)

openai = await client.get_openai_client(api_version="2024-02-01")

# Test with the same strings from your APIM data
# text1 = "summarize history of Poland"
# text2 = "write a poem about turtles, make it nice for kids, 2 paragraphs, make it silly"
text1 = "what's the impact of AI on healthcare?"
text2 = "explain the rules of cricket?"
embedding1 = await get_embedding(text1, openai, model="text-embedding-ada-002")
embedding2 = await get_embedding(text2, openai, model="text-embedding-ada-002")

print("Testing different similarity approaches between the two embeddings.")
print(f"\tText 1: {text1}")
print(f"\tText 2: {text2}")
print("=" * 50)

# Method 1: Raw cosine similarity (might be negative)
raw_cosine = cosine_similarity_raw(embedding1, embedding2)
print(f"Raw cosine similarity: {raw_cosine:.8f}")
print(f"Raw cosine distance: {1 - raw_cosine:.8f}")
print(f"Sum: {raw_cosine + (1 - raw_cosine):.8f}")
print()

# Method 2: Clipped cosine similarity (no negative values)
clipped_cosine = max(0, raw_cosine)
print(f"Clipped cosine similarity: {clipped_cosine:.8f}")
print(f"Clipped cosine distance: {1 - clipped_cosine:.8f}")
print(f"Sum: {clipped_cosine + (1 - clipped_cosine):.8f}")
print()

# Method 3: Standard cosine distance
cos_distance = cosine_distance(embedding1, embedding2)
cos_similarity = 1 - cos_distance
print(f"Cosine distance approach - similarity: {cos_similarity:.8f}")
print(f"Cosine distance approach - distance: {cos_distance:.8f}")
print(f"Sum: {cos_similarity + cos_distance:.8f}")
print()

# APIM expected values for comparison:
# similarity: 0.720192, distance: 0.27980798
print("APIM expected values:")
print("Similarity: 0.720192, Distance: 0.27980798")
print()

# Use the method that seems closest to APIM
similarity = cos_similarity  # This should be closest to APIM
distance = cos_distance

# Format output like APIM semantic cache
threshold = 0.8

if distance < threshold:
    print(
        f"Found cache entry using semantic search within threshold of '{threshold}', and similarity score '{similarity:.6f}' under the vary-by partition 'None'. \nVector distance is '{distance:.8f}'."
    )
    print("\ncosine_distance = 1 - cosine_similarity.")
    print(f"cosine_distance '{distance:.8f}' was under the threshold of '{threshold}'.")
    print(
        f"\ncosine_distance '{distance:.8f}' measures how far apart two vectors are. Approaching 0 means very close match, approaching 1 means weak match."
    )
else:
    print(
        f"No cache entry found using semantic search within threshold of '{threshold}'. Similarity score '{similarity:.6f}' under the vary-by partition 'None'. \nVector distance is '{distance:.8f}'."
    )
    print("The texts are not similar.")